In [1]:
%load_ext autoreload
%autoreload 2

from src.model.news_encoder import LookupNewsEncoder, NEConfig
from src.model.user_encoder import (
    NewsSelfAttention,
    PopularityEmbedding,
    PopularityAwareUserEncoder,
    ContentPopularityJointAttention,
    CPJAConfig,
    PEConfig,
    NSAConfig,
    PAUEConfig,
)
from src.model.popularity_predictor import TimeAwareNewsPopularityPredictor, TANPPConfig

from src.data.split import EBNeRDSplit

import torch

Some data for testing


In [2]:
split = EBNeRDSplit()

Settings defined in the paper

In [3]:
popularity_embedding_size = 100
recency_embedding_size = 100
n_attention_heads = 20
head_output_size = 20

Settings found in their code

In `Encoders.py`, function `create_pe_model`, lines creating variables `time_embedding_layer` and `population_embedding_layer`. Don't fully get it all though.


In [4]:
max_ctr = 200
max_recency = 1500

In `Main.ipynb`.

In [5]:
max_clicked = 50

# News Encoder


In [6]:
news_encoder_config = NEConfig(
    n_attention_heads=n_attention_heads,
    head_output_size=head_output_size,
)
lookup_news_encoder = LookupNewsEncoder("bert", news_encoder_config, device=torch.device("cpu"))
print(lookup_news_encoder)
print(lookup_news_encoder.__doc__)
print(lookup_news_encoder.forward.__doc__)

LookupNewsEncoder(
  (fcout): Linear(in_features=768, out_features=400, bias=True)
)


    Songga told us to first use the premade article embedding in one
    of the artifacts as a newencoder.

    So this news encoder lookes up the embeddings of the articles in
    the data artifact and convert them to the desired size, using a
    fully connected layer.

    


        Returns n (the news encodings) for a batch of articles.

        n has shape (batch_size, embedding_size),

        


In [7]:
random_article_id = split.get_random_article_id()
news_embeddings = lookup_news_encoder.forward([random_article_id])
news_embeddings.shape

torch.Size([1, 400])

# User Encoder


### News Self Attention


In [14]:
news_self_attention_config = NSAConfig(
    n_size=news_encoder_config.get_size_n(),
    n_attention_heads=n_attention_heads,
    head_output_size=head_output_size,
    max_clicked=max_clicked,
)
news_self_attention = NewsSelfAttention(news_self_attention_config)
print(news_self_attention)
print(news_self_attention.__doc__)
print(news_self_attention.forward.__doc__)

NewsSelfAttention()


    Implementation of the news self-attention module for
    the popularity-aware user encoder.

    


        Calculates the contextual news representations m, based on the
        news embeddings n outputted by the news encoder.

        n is a tensor of shape (batch_size, max_clicked, news_embedding_size)
        m is a tensor of shape (batch_size, max_clicked, head_output_size * n_attention_heads)
        where max_clicked is the number of clicked articles by the user.

        


In [15]:
n = news_embeddings.unsqueeze(1).repeat(
    1,
    max_clicked,
    1,
)
contextual_news_embeddings = news_self_attention.forward(n)
contextual_news_embeddings.shape

torch.Size([1, 50, 400])
50 50


torch.Size([1, 50, 400])

### Popularity Embedding


In [52]:
popularity_embedding_config = PEConfig(
    p_size=popularity_embedding_size, max_ctr=max_ctr, max_clicked=max_clicked
)
popularity_embedding = PopularityEmbedding(popularity_embedding_config)
print(popularity_embedding)
print(popularity_embedding.__doc__)
print(popularity_embedding.forward.__doc__)

PopularityEmbedding(
  (embedding): Embedding(200, 100)
)


    Implementation of the popularity embedding module for the popularity-aware user encoder.

    "Second, we uniformly quantify the popularity of the i-th clicked news predicted by
    the time-aware news popularity predictor and convert it into an embedding vector $p_i$
    via popularity embedding."

    They also leave a footnote, right after 'popularity predictor'.

    "We remove news recency and content here to avoid non-differentiable quantization operation."

    I don't really get this, how is the does anything here. To me it just seems like we have
    to use the click though rates with this Embedding layer.

    Q: What has the popularity predictor to do with the popularity embedding?

    


        Calculates the popularity embeddings p based on the click through rate
        of every article.

        ctr is a tensor of shape (batch_size, N), where N is the number
        of clicked articles by the user. The val

In [54]:
popularity_scores = torch.randint(0, max_ctr, (1, 1))
popularity_embeddings = popularity_embedding.forward(popularity_scores)
popularity_embeddings.shape

torch.Size([1, 1, 100])

### Content Popularity Join Attention


In [55]:
cpja_config = CPJAConfig(
    p_size=popularity_embedding_config.p_size,
    m_size=news_self_attention_config.get_size_m(),
    # The paper doesnt specify a default I think
    weight_size=100,
    max_clicked=max_clicked,
)
cpja = ContentPopularityJointAttention(cpja_config)
print(cpja)
print(cpja.__doc__)
print(cpja.forward.__doc__)

ContentPopularityJointAttention()


    Implementation of the content-popularity joint attention module
    for the popularity-aware user encoder.

    This is based on formula (2) in 3.4 of the paper.

    


        Calculates the user interest embeddings u, based on the
        the popularity embeddings p, and the contextual news
        representations m.

        m is a tensor of shape (batch_size, N, m_size)
        p is a tensor of shape (batch_size, N, p_size)
        u is a tensor of shape (batch_size, m_size)
        where N is the number of clicked articles by the user.

        


In [56]:
user_interests = cpja.forward(contextual_news_embeddings, popularity_embeddings)
user_interests.shape

AssertionError: 

# Popularity Predictor

In [59]:
time_aware_news_popularity_predictor_config = TANPPConfig(
    r_size=recency_embedding_size,
    n_size=news_encoder_config.get_size_n(),
    max_recency=max_recency,
)

time_aware_news_populariry_predictor = TimeAwareNewsPopularityPredictor(
    time_aware_news_popularity_predictor_config
)